In [25]:
import advent
from typing import Literal
data = advent.get_char_grid(17, 'csv', int)
SIZE = data.shape


In [26]:
# Maze time! I will use my pre-made dijkstra algorithm
# This requires an 'adjacent' function:

DIRECTION = Literal["north", "west", "east", "south"]
TRAVEL = int # for part 1 can only be 0, 1, 2, 3, for part 2 can be up to 10
NODE = tuple[tuple[int, int], TRAVEL, DIRECTION]
ALL_DIRECTIONS: list[DIRECTION] = ["north", "west", "east", "south"]
reverse: dict[DIRECTION, DIRECTION] = {'north': 'south', 'west': 'east', 'south': 'north', 'east': 'west'}

def is_target(node: NODE):
    return node[0] == (SIZE[0]-1, SIZE[1]-1)

def walkable(coord: tuple[int, int]):
    return coord[0] >= 0 and coord[1] >= 0 \
        and coord[0] < SIZE[0] and coord[1] < SIZE[1]

def tadd(a: tuple[int, int], b: tuple[int, int]):
    return a[0] + b[0], a[1] + b[1]

def step(coord: tuple[int, int], direction: DIRECTION) -> tuple[int, int]:
    match direction:
        case 'north': return tadd(coord, (-1, 0))
        case 'east': return tadd(coord, (0, 1))
        case 'west': return tadd(coord, (0, -1))
        case 'south': return tadd(coord, (1, 0))

def allowed_directions_part1(node: NODE) -> list[DIRECTION]:
    _, travel, direction = node
    allowed_directions = ALL_DIRECTIONS.copy()
    allowed_directions.remove(reverse[direction])
    if travel == 3: allowed_directions.remove(direction)
    return allowed_directions

def allowed_directions_part2(node: NODE) -> list[DIRECTION]:
    _, travel, direction = node
    # REALLY ANNOYING HACKY EDGECASE. this only comes up during the start node
    if travel == 0: return ['east', 'south']
    # Minimum travel distance
    if travel < 4: return [direction]
    allowed_directions = ALL_DIRECTIONS.copy()
    allowed_directions.remove(reverse[direction])
    if travel == 10: allowed_directions.remove(direction)
    return allowed_directions

# Maze representation: each 'node' consists of a:
# - coordinate (x, y)
# - how long we've been traveling in this direction
# - the direction we've been traveling in
# logic:
# if TRAVEL is 3, we MUST change direction
# whenever you change direction, TRAVEL goes to 1
# DIRECTION must never reverse 180 degrees (e.g. west->east is not allowed)
# SIZE is a global variable containing grid size
def adjacent(node: NODE, part1=True) -> list[tuple[NODE, int]]:
    coord, travel, direction = node

    if part1:
        allowed_directions = allowed_directions_part1(node)
    else:
        allowed_directions = allowed_directions_part2(node)

    adjacent: list[tuple[NODE, int]] = []
    for new_direction in allowed_directions:
        new_coord = step(coord, new_direction)
        if not walkable(new_coord): continue
        cost = data[new_coord]
        new_travel = travel + 1 if new_direction == direction else 1
        node: NODE = (new_coord, new_travel, new_direction)
        adjacent.append((node, cost))
    return adjacent



In [23]:
from advent.maze import solve_maze

# In the beginning, set travel to 0 so we can still set 3 steps east
start_node: NODE = ((0, 0), 0, 'east')
total_nodes = SIZE[0] * SIZE[1] * 3 * 4 # 3 travel values, 4 directions
# runtime: 1 minute
result = solve_maze(start_node, is_target, adjacent, None, total_nodes)

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 1138


In [27]:
def part2_is_target(node: NODE):
    return node[0] == (SIZE[0]-1, SIZE[1]-1) and node[1] >= 4
part2_adjacent = lambda node: adjacent(node, False)

start_node: NODE = ((0, 0), 0, 'east')
total_nodes = SIZE[0] * SIZE[1] * 10 * 4 # 10 travel values, 4 directions
# runtime: 10 minutes
result = solve_maze(start_node, part2_is_target, part2_adjacent, None, total_nodes)

  0%|          | 0/1 [00:00<?, ?it/s]

Final path length: 1312
